# Merton Jump Diffusion: Option Pricing with Monte Carlo

In this notebook, we show how to use Monte Carlo simulation to price european options with a Merton jump diffusion process. The implementation is very similar to the one using the Black-Scholes model and the Euler-Maruyama scheme. The key differences are the additional term added to the risk-neutral drift and the normal compound Poisson process added to the geometric Brownian motion.

In [3]:
import numpy as np

# Market parameters
T = 1      # maturity
S0 = 1     # spot price
K = 1.1    # strike price
r = 0.05   # risk-free interest rate
q = 0.02   # dividend rate

# Model parameter
sigma = 0.4 # volatility

# Model parameters of the jump part (NCPP)
mu_j = -0.1
sigma_j = 0.15
lmbda = 0.5

# Risk-neutral measure
muRN = r-q-0.5*sigma**2 - lmbda*(np.exp(mu_j+0.5*sigma_j**2)-1) # drift

# Monte Carlo parameters; npaths = nblocks*nsample
nblocks = 20000 # number of blocks
nsample = 10000 # number of samples per block

# Initialize arrays
Vc_list = np.zeros(nblocks) # call array
Vp_list = np.zeros(nblocks) # put array

# Monte Carlo
for i in range(nblocks):
    A = muRN*T + sigma*np.sqrt(T) * np.random.normal(size=(1, nsample))
    N = np.random.poisson(lmbda, size=(1, nsample))
    J = mu_j* N + sigma_j * np.sqrt(N) * np.random.normal(size=(1, nsample))
    X = A+J
    S = S0*np.exp(X)
    Vc_list[i] = np.exp(-r*T)*np.mean(np.maximum(S - K, 0))
    Vp_list[i] = np.exp(-r*T)*np.mean(np.maximum(K - S, 0))

# Final call and put values
Vc = np.mean(Vc_list)
Vp = np.mean(Vp_list)

print('Call price: ' + str(round(Vc, 4)))
print('Put price:  ' + str(round(Vp, 4)))

Call price: 0.1362
Put price:  0.2023
